In [1]:
import glob
import os
import pandas as pd

from ase.io import read, write
from ase.optimize import BFGS
from ase.filters import UnitCellFilter
from tensorpotential.calculator import TPCalculator

# --- CONFIG ---
poscar_pattern = "../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/*"   
fmax = 0.02
csv_out = "results.csv"

[tensorpotential] Info: Environment variable TF_USE_LEGACY_KERAS is automatically set to '1'.


2026-01-29 16:16:18.448227: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-29 16:16:18.461768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769670978.477920   30934 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769670978.482458   30934 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769670978.495247   30934 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
from tensorpotential.calculator import grace_fm
calc = grace_fm('GRACE-2L-OMAT')

Using cached GRACE model from /home/vsbat/.cache/grace/GRACE-2L-OMAT
Model license: Academic Software License


I0000 00:00:1769670993.088136   30934 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6156 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
# calc = grace_fm('GRACE-1L-OMAT')  
# ['GRACE-1L-MP-r6', 'GRACE-2L-MP-r5', 'GRACE-2L-MP-r6', 'GRACE-FS-OAM', 'GRACE-1L-OAM', 'GRACE-2L-OAM', 'GRACE-FS-OMAT', 'GRACE-1L-OMAT', 'GRACE-2L-OMAT'] 
model_path = "/home/vsbat/.cache/grace/GRACE-2L-OMAT" #'GRACE-2L-OAM' -- 

In [8]:
# --- init calculator ---
# calc = TPCalculator(model_path)

results = []

for poscar_file in glob.glob(poscar_pattern):
    print(f"==> Traitement {poscar_file}")
    try:
        # lire
        atoms = read(poscar_file, format="vasp")
        atoms.set_calculator(calc)

        # énergie initiale
        E_init = atoms.get_potential_energy()

        # relaxer (cellule + positions)
        atoms_ucf = UnitCellFilter(atoms)
        dyn = BFGS(atoms_ucf, logfile="relax.log")  
        dyn.run(fmax=fmax)

        # énergie finale
        E_final = atoms.get_potential_energy()

        # sauvegarder maille relaxée à côté
        #out_file = os.path.join(os.path.dirname(os.path.dirname(poscar_file), "POSCAR_relax")
        out_file = os.path.splitext(poscar_file)[0] + "_relax.cif"
        write(out_file, atoms, format="cif")

        # stocker résultats
        results.append({
            "file": poscar_file,
            "E_init (eV)": E_init,
            "E_final (eV)": E_final,
            "deltaE (eV)": E_final - E_init
        })

    except Exception as e:
        print(f"Erreur sur {poscar_file}: {e}")

# --- écrire CSV ---
df = pd.DataFrame(results)
df.to_csv(csv_out, index=False)
print(f"Résultats écrits dans {csv_out}")

==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm006223869-Co46_POSCAR


/tmp/ipykernel_26598/3032607643.py:11: FutureWarning: Please use atoms.calc = calc
  atoms.set_calculator(calc)
I0000 00:00:1769670111.447154   26598 service.cc:152] XLA service 0x460042a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769670111.447207   26598 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2026-01-29 16:01:51.704666: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769670112.259648   26598 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1769670116.798526   26598 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm002156187-Li4Co2_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm004447020-Co8_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm070902686-Co4O4_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm005423490-O6_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm003160988-Li3_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm003239050-Li6Co8O16_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm005436743-Li1Co5_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm005514569-Li8Co4_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCARS/agm004447026-Co3_POSCAR
==> Traitement ../../materials_dataset/unittests_datasets/alex_LiCoO_POSCAR

KeyboardInterrupt: 